<a href="https://colab.research.google.com/github/boyerb/Investments/blob/master/Ex10-TestingCAPM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Investment Analysis**, Bates, Boyer, Fletcher

# Example Chater 10: Testing the CAPM
In this notebook we download return data from Ken French's website and estimate the alpha's for each porfolio along with the 95% confidence intervals.   

### Imports and Setup

In [ ]:
!curl -O https://raw.githubusercontent.com/boyerb/Investments/master/functions/simple_finance.py
import simple_finance as sf
import pandas as pd

### Choose Strategy and Estimate Alpha

**`get_ff_strategies(stype: string, start_date: string, end_date: string, details: bool = False)-> pd.DataFrame`**     
Imports historical monthly returns for decile portfolios based on trading strategies from Ken French's website and returns the data as a DataFrame  

**Inputs**  
- `stype` (*str*):  strategy type, can be either `'beta'` or `'momentum'`. Other strategies coming soon.  
- `start_date` (*str*, optional):  starting month in format `'yyyy-mm'`  
- `end_date` (*str*, optional): ending month in format `'yyyy-mm'`
- `details`(*bool*, optional, default=`False`):  
  - If True, the function will print out a simple description of the strategy and construction details.  

**Returns**  
- (*DataFrame*): Decile returns for the trading stratgey, the value-weighted market return, and the return on short-term t-bills.
  - *Index*: Monthly date index (`pandas.PeriodIndex` with `freq='M'`).
 - *Decile Returns for Strategy* : Dec 1, Dec 2, Dec 3,...Dec 9, Dec 10
 - *mkt* : The return on the value-weighted portfolio
 - *rf* : The return on short-term t-bills

**Example Usage**:  
1. `dat=sf.get_ff_strategies(stype='momentum')`  
Imports all avaiable monthly returns for the momentum strategy.

2. `dat=sf.get_ff_strategies(stype='beta', start_date='1990-01', end_date=`2024-12`, details=True)`  
Imports monthly returns for the beta strategy from January 1990 to December 2024, and prints out information about the strategy.


In [ ]:
# Choose the strategy
dat=sf.get_ff_strategies(stype='momentum', start_date='2021-01', end_date='2024-12')

dat[exret]=dat

# Estimate the alpha
X=df[['Tech Portfolio', 'Oil Portfolio']]
Y=df['Google']
X=sm.add_constant(X) # we specify that we want to add a constant to the regression equation
model=sm.OLS(Y,X).fit()
params = model.params
conf_int = model.conf_int(alpha=0.05)  # 95% CIs
r_squared = model.rsquared
# create a table of output
results_df = pd.DataFrame({
    'Parameter': params.index, # 1st column: parameter name
    'Estimate': params.values, # 2nd column: parameer value
    'CI Lower': conf_int[0].values, # 3rd column: lower bound on 95\% CI
    'CI Upper': conf_int[1].values  # 4th column: upper bound on 95\% CI
})

print(results_df)
print()
print("R-squared", r_squared)
